In [1]:
import pandas as pd
from datetime import datetime
from sqlalchemy import create_engine

In [2]:
# this is to make it so that new files can be easily analyzed
first_date = '04-12-2020'
end_date = '02-02-2021'

# create date range and format it nicely
dates = pd.date_range(first_date, end_date)
dates.format(formatter=lambda x: x.strftime('%m-%d-%Y'))

date_list = []
# print all dates
for date in dates:
    current_date = date.strftime('%m-%d-%Y')
    #print(current_date)
    date_list.append(current_date)

print("Date list has been generated, first date: " + str(date_list[0]) + ", last date: " + str(date_list[-1]) + ", number of dates: " + str(len(date_list)))

Date list has been generated, first date: 04-12-2020, last date: 02-02-2021, number of dates: 297


In [3]:
# This is because SQLite3 takes ISO-8061 date formats

iso_dates_list = []

for date in date_list:
    datetime_obj = datetime.strptime(date, '%m-%d-%Y')
    #print(datetime_obj.isoformat())
    iso_dates_list.append(datetime_obj.isoformat())

In [6]:
# start sqlite3 database in RAM
engine = create_engine('sqlite://', echo=False)

In [7]:
# clean and prepare data for database

path = r'COVID-19/csse_covid_19_data/csse_covid_19_daily_reports_us/'

for i in range(len(date_list)):
    # load data into dataframe
    data = pd.read_csv(path + date_list[i] + ".csv")

    # clean column name
    data.columns = data.columns.str.replace(' ', '')

    # add date column
    data.insert(2, "Date", iso_dates_list[i], True)

    # change names which for some reason change from 2020 to 2021
    data.rename(columns={"Total_Test_Result": "People_Tested", "Case_Fatality_Ratio": "Mortality_Rate"})

    # load dataframe into table
    data.to_sql('Covid-Data', con=engine, if_exists='append', chunksize=1000)

OperationalError: (sqlite3.OperationalError) table Covid-Data has no column named Total_Test_Results
[SQL: INSERT INTO "Covid-Data" ("index", "Province_State", "Country_Region", "Date", "Last_Update", "Lat", "Long_", "Confirmed", "Deaths", "Recovered", "Active", "FIPS", "Incident_Rate", "Total_Test_Results", "People_Hospitalized", "Case_Fatality_Ratio", "UID", "ISO3", "Testing_Rate", "Hospitalization_Rate") VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)]
[parameters: ((0, 'Alabama', 'US', '2020-11-09T00:00:00', '2020-11-10 05:42:01', 32.3182, -86.9023, 206523, 3084, 84471.0, 117302.0, 1.0, 4178.039376446126, 1406829.0, None, 1.5054403803628873, 84000001, 'USA', 28692.146023452104, None), (1, 'Alaska', 'US', '2020-11-09T00:00:00', '2020-11-10 05:42:01', 61.3707, -152.4044, 20300, 84, 7161.0, 13055.0, 2.0, 2774.948909499757, 767997.0, None, 0.4137931034482759, 84000002, 'USA', 104982.87870192537, None), (2, 'American Samoa', 'US', '2020-11-09T00:00:00', '2020-11-10 0

In [ ]:
engine.execute("SELECT * FROM Covid-Data").fetchall()